<a href="https://colab.research.google.com/github/sabumjung/Pytorch/blob/master/AutoEncoder2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import sys
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/98_module/')

In [0]:
from m_plot import plot
from m_data import gn
from m_mdl import classify, parameter_opt
import numpy as np

In [0]:
x=np.arange(28)
y=np.arange(28)
a1=plot('scatterplot', x, y)
a1.pltshow()

In [6]:
from m_DLmdl import model_AE
from keras.datasets import mnist, fashion_mnist
import numpy as np
import os
import argparse
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import math

# curdir = os.path.dirname(os.path.abspath(__file__))
# parser = argparse.ArgumentParser()
# parser.add_argument('--optimizer', choices=['adam', 'sgd', 'adagrad'], default='adam')
# parser.add_argument('--loss', choices=['mean_squared_error', 'binary_crossentropy'], default='mean_squared_error')
# parser.add_argument('--epochs', type=int, default=10)
# parser.add_argument('--batch_size', type=int, default=64)
# parser.add_argument('--test_samples', type=int, default=50)
# parser.add_argument('--result', default=os.path.join(curdir, 'result.png'))

class in_args:
    def __init__(self, optimizer='adam', loss='mean_squared_error', epochs=10, batch_size=64, test_samples=50, result='result.png'):
        self.optimizer=optimizer   # adam, sgd, adagrad
        self.loss=loss             # mean_squared_error, binary_crossentropy
        self.epochs=epochs
        self.batch_size=batch_size
        self.test_samples=test_samples
        self.result=result


def main(a3):
    # prepare normal dataset (Mnist)
    (x_train, _), (x_test, _) = mnist.load_data()
    x_train = x_train / 255.  # normalize into [0,1]
    x_test = x_test / 255.

    # prapare abnormal dataset (Fashion Mnist)
    (_, _), (x_abnormal, _) = fashion_mnist.load_data()
    x_abnormal = x_abnormal / 255.

    # sample args.test_samples images from eaech of x_test and x_abnormal
    perm = np.random.permutation(a3.test_samples)
    x_test = x_test[perm][:a3.test_samples]
    x_abnormal = x_abnormal[perm][:a3.test_samples]

    # train each model and test their capabilities of anomaly deteciton
    model_names = ['autoencoder', 'deep_autoencoder', 'convolutional_autoencoder']
    
    a1=plot('linechart')    
    
    for model_name in model_names:
        # instantiate model
        model = model_AE(model_name).load_model()

        # reshape input data according to the model's input tensor
        if model_name == 'convolutional_autoencoder':
            x_train = x_train.reshape(-1, 28, 28, 1)
            x_test = x_test.reshape(-1, 28, 28, 1)
            x_abnormal = x_abnormal.reshape(-1, 28, 28, 1)
        elif model_name == 'autoencoder' or model_name == 'deep_autoencoder':
            x_train = x_train.reshape(-1, 28 * 28)
            x_test = x_test.reshape(-1, 28 * 28)
            x_abnormal = x_abnormal.reshape(-1, 28 * 28)
        else:
            raise ValueError('Unknown model_name %s was given' % model_name)

        # compile model
        model.compile(optimizer=a3.optimizer, loss=a3.loss)

        # train on only normal training data
        model.fit(
            x=x_train,
            y=x_train,
            epochs=a3.epochs,
            batch_size=a3.batch_size,
        )

        # test
        x_concat = np.concatenate([x_test, x_abnormal], axis=0)
        losses = []
        for x in x_concat:
            # compute loss for each test sample
            x = np.expand_dims(x, axis=0)
            loss = model.test_on_batch(x, x)
            losses.append(loss)
        plt.plot(range(len(losses)), losses, linestyle='-', linewidth=1, label=model_name)

        # delete model for saving memory
        del model

    # create graph
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('sample index')
    plt.ylabel('loss')
    plt.show()
    plt.savefig(a3.result)
    plt.clf()


if __name__ == '__main__':
    a3=in_args(result='/content/gdrive/My Drive/Colab Notebooks/result.png', epochs=10, batch_size=64, test_samples=500)
    main(a3)

Epoch 1/10
60000/60000 [==============================] - 4s 65us/step - loss: 0.0325
Epoch 2/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.0107
Epoch 3/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.0066
Epoch 4/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.0052
Epoch 5/10
60000/60000 [==============================] - 4s 59us/step - loss: 0.0047
Epoch 6/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.0044
Epoch 7/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.0043
Epoch 8/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.0042
Epoch 9/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.0041
Epoch 10/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.0041
Epoch 1/10
60000/60000 [==============================] - 7s 114us/step - loss: 0.0295
Epoch 2/10
60000/60000 [============================

<Figure size 432x288 with 0 Axes>